## Start of project

In [1]:
!pip install -U spacy
!pip install -U matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip show matplotlib

Name: matplotlib
Version: 3.6.2
Summary: Python plotting package
Home-page: https://matplotlib.org
Author: John D. Hunter, Michael Droettboom
Author-email: matplotlib-users@python.org
License: PSF
Location: /usr/local/lib/python3.8/dist-packages
Requires: pyparsing, pillow, cycler, numpy, python-dateutil, fonttools, packaging, kiwisolver, contourpy
Required-by: yellowbrick, wordcloud, seaborn, scikit-image, pycocotools, prophet, plotnine, pandas-profiling, mlxtend, mizani, missingno, matplotlib-venn, keras-vis, imgaug, fastai, descartes, datascience, daft, arviz


In [3]:
!python -m spacy download en_core_web_lg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 587.7 MB 16 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [4]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import random
random.seed(12)
import spacy
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import pandas as pd
import torch.nn.functional as F

from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


## **PART 1: LSTM FOR LANGUAGE MODELING**

**Task 1:** Follow the tutorial in [here](https://www.analyticsvidhya.com/blog/2020/08/build-a-natural-language-generation-nlg-system-using-pytorch/) to train a word-level LSTM language model. Train the language model on texts
from the file prideAndPrejudice.txt. Before using it to train the language model, you need to first sentence-segment, then
tokenize, then lower case each line of the file using Spacy. Append start-of-sentence token ’\<s\>’ and end-of-sentence
’\</s\>’ token to each sentence and put each sentence in its own line. Use only words that appear more than once in this
corpus and assign UNK tokens for the rest; you may also need to pad sentences that are shorter than 5 (see [here](https://github.com/gabrielloye/LSTM_Sentiment-Analysis/blob/master/main.ipynb) in cell
10-12 for adding unknown: UNK token and padding: PAD token to your vocabulary). Train the language model and
save the trained model (see [here](https://pytorch.org/tutorials/beginner/saving_loading_models.html)). Generate 10 examples of text from it, starting from ’\<s\>’ token and ending at ’\</s\>’
token.

**Checkpoint 1:**

Load texts from by lines. The samples of your loaded text lines should look like the following.

In [124]:
import requests
from bs4 import BeautifulSoup

data = []

# Make a request
# https://awoiaf.westeros.org/index.php/Winterfell
# page = requests.get('https://www.gutenberg.org/cache/epub/3811/pg3811-images.html')
# page = requests.get('https://en.wikipedia.org/wiki/The_Killer_Angels')
page = requests.get('http://www.script-o-rama.com/movie_scripts/g/gettysburg-script-transcript-civil-war.html')
soup = BeautifulSoup(page.content, 'html.parser')

data.append({
    'book': soup.h2.get_text(),
    'table' : soup.find("pre").find(text=True),
    'text': ' '.join([p.get_text(strip=True) for p in soup.select('body p')[2:]])
    }
)
getty = [data[0]['table']]

In [5]:
def findFiles(path): return glob.glob(path)

# Build the category_lines dictionary, a list of names per language
all_categories = []

# Read a file and split into lines
def readLines(filename):
    lines = pd.read_csv(filename, encoding='utf-8')
    return lines

for filename in findFiles('/content/drive/MyDrive/Colab Notebooks/CS_505_rs_prjt/Game*.csv'):
    novel = os.path.splitext(os.path.basename(filename))
    all_categories.append(novel)
    df_lines = readLines(filename)
    print(df_lines)

print(novel)

      Release Date    Season    Episode     Episode Title              Name  \
0       2011-04-17  Season 1  Episode 1  Winter is Coming      waymar royce   
1       2011-04-17  Season 1  Episode 1  Winter is Coming              will   
2       2011-04-17  Season 1  Episode 1  Winter is Coming      waymar royce   
3       2011-04-17  Season 1  Episode 1  Winter is Coming              will   
4       2011-04-17  Season 1  Episode 1  Winter is Coming             gared   
...            ...       ...        ...               ...               ...   
23906   2019-05-19  Season 8  Episode 6   The Iron Throne           brienne   
23907   2019-05-19  Season 8  Episode 6   The Iron Throne             bronn   
23908   2019-05-19  Season 8  Episode 6   The Iron Throne  tyrion lannister   
23909   2019-05-19  Season 8  Episode 6   The Iron Throne               man   
23910   2019-05-19  Season 8  Episode 6   The Iron Throne               all   

                                                Sen

In [6]:
df_lines

n = 5
df_lines.sample(n, random_state=12)

,Release Date,Season,Episode,Episode Title,Name,Sentence
5142,2012-05-06,Season 2,Episode 6,The Old Gods and the New,osha,Let me serve you.
17489,2016-05-01,Season 6,Episode 2,Home,missandei,They are not eating They haven't touched any f...
3760,2012-04-08,Season 2,Episode 2,The Night Lands,tyrion lannister,The fool! He had no idea you were already bought.
9744,2013-05-19,Season 3,Episode 8,Second Sons,stannis baratheon,So did Renly.
5422,2012-05-06,Season 2,Episode 6,The Old Gods and the New,jaqen hghar,A girl cannot tell a man when exactly he must ...


In [7]:
lines_dict = dict(enumerate(df_lines.Sentence.values))
print(lines_dict)
print(len(lines_dict))


{0: "What do you expect? They're savages. One lot steals a goat from another lot and before you know it, they're ripping each other to pieces.", 1: "I've never seen wildlings do a thing like this. I've never seen a thing like this, not ever in my life.", 2: 'How close did you get?', 3: 'Close as any man would.', 4: 'We should head back to the wall.', 5: 'Do the dead frighten you?', 6: "Our orders were to track the wildlings. We tracked them. They won't trouble us no more.", 7: "You don't think he'll ask us how they died? Get back on your horse.", 8: 'Whatever did it to them could do it to us. They even killed the children.', 9: "It's a good thing we're not children. You want to run away south, run away. Of course, they will behead you as a deserter … If I don't catch you first. Get back on your horse. I won't say it again.", 10: 'Your dead men seem to have moved camp.', 11: 'They were here.', 12: 'See where they went.', 13: 'What is it?', 14: "It's …", 15: "Go on. Father's watching.", 

In [98]:
import re
import spacy
from tqdm import tqdm
def remove_extras(in_texts):
    textList = []
    for in_text in in_texts:
        t_en = in_text.encode("ascii", "ignore")
        t = t_en.decode()
        t = re.sub("\n", " ", t)
        t = re.sub("\r", " ", t)
        t = re.sub("&lt;", "<", t)
        t = re.sub("&gt;", ">", t)
        t = re.sub("&amp;", "&", t)
        t = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', t, flags=re.MULTILINE) # URLs
        t = re.sub(r'@[\w]+','', t, flags=re.MULTILINE) # mentions
        t = re.sub(r'#[\w]+','', t, flags=re.MULTILINE) # hashtags
        t = re.sub("[!@#$%^&*()[]{};:,/<>\|`~-=_+]", " ", t)
        t = re.sub(r"([.!?])", r" \1", t)
        textList.append(t)
    return textList

In [127]:
getty = remove_extras(getty)
getty[0][:1300]

'                          In June 1863 after more than two years  of bloody conflict   .   .   .                            the Confederate army of Northern  Virginia, Robert E   . Lee commanding   .   .   .                            slips across the Potomac  to begin the invasion of the North   .                            It is an army of       men   .                            They move slowly behind the Blue Ridge   .   .   .                            using the mountains  to screen their movements   .                            Their objective is to draw  the Union army out into the open   .   .   .                            where it can be destroyed   .                            Late in June, the Union army  of the Potomac,       men   .   .   .                             turns north from Virginia to begin  the great pursuit up the narrow roads   .   .   .                             across Maryland and into Pennsylvania   .                             General Lee knows tha

**Checkpoint 2:**

Preprocess your loaded texts with spacy, also convert words into lower-case, and also pad sentences with \<s\> and \</s\>. The preprocessed lines should look like the following.

In [8]:
def preprocess(textList, batchsz=500):
    # Input: text separated by new lines
    # Output: sentences with space around punctuation

    # sub-task 1: segment all the sentences in the wiki texts.
    # sub-task 2: tokenize the sentences from sub-task 2.
    # sub-task 3: lower-case the tokens from sub-task 3/4.

    nlp = spacy.load('en_core_web_lg')
    docDict = {}
    docList = []
    dropped = []
    cnt = 0

    for doc in tqdm(nlp.pipe(textList, batch_size=batchsz)): # Step 1
        # if doc.has_annotation("DEP"):
        # for ent in doc.ents:
            #  THIS LINE GIVES THE NAMED ENTITIES IN DOC
            # print(ent.text, ent.start_char, ent.end_char, ent.label_)
        for spanText in doc.sents:
            spanT = spanText.text.strip()
            if len(spanT) < 8:
                dropped.append(spanT)
                continue
            if spanT:
                docString = '<s> ' 
                for tok in spanText:
                    docString += tok.text.strip().lower() + ' ' # Step 2 & 3
                docString += ' </s>'
                docDict[cnt]= docString
                docList.append(docString)
                cnt += 1
    return  docDict, docList, dropped

In [9]:
sentDict, sent_list, dropped = preprocess(df_lines.Sentence.values)
print(f" \nline size: {len(sentDict)}")
print(sent_list[0:2])

23911it [00:47, 500.33it/s]

 
line size: 40750
['<s> what do you expect ?  </s>', "<s> they 're savages .  </s>"]


In [100]:
gettyDict, getty_list, dropped = preprocess(getty)
print(f" \nline size: {len(getty_list)}")
print(sent_list[0:2])

1it [00:05,  5.13s/it]

 
line size: 3426
['<s> what do you expect ?  </s>', "<s> they 're savages .  </s>"]


**Checkpoint 3:**

Define a function to prepare fixed-length sequences from our dataset. Here we specified the length of the sequence as five. You may refer to the tutorial to see how this can be done. Remember to deal with the case of tokens of length less or equal to 5. The following example show what the fixed-length sequence should look like given  input texts. Also compare the size of the processed sequences to see if the number is close.

NOTE: the example is just the processed result of one line, your result sequences should contain processed results for all lines.

In [101]:
# create sequences of length 5 tokens
def create_seq(text, seq_len = 5):
    
    sequences = []

    # if the number of tokens in 'text' is greater than 5
    if len(text.split()) > seq_len:
        for i in range(seq_len, len(text.split())):
            # select sequence of tokens
            seq = text.split()[i-seq_len:i+1]
            # add to the list
            sequences.append(" ".join(seq))

        return sequences

    # if the number of tokens in 'text' is less than or equal to 5
    else:
        # select sequence of tokens
        seq = text.split()
        for i in range(5-len(seq)+1):
            seq.append('_PAD')
        # add padding to the list to fill out all 5 positions
        sequences.append(" ".join(seq))

        return sequences

In [11]:
# sample the original sentence
print(sent_list[123])

seqs = [create_seq(i) for i in sent_list if i != '_']

# sample the sentence separated text
print(seqs[123])

# merge list-of-lists into a single list
seqs = sum(seqs, [])

# count of sequences
print(f'lines size: {len(sent_list)} \nseq size: {len(seqs)}')

<s> my husband .  </s>
['<s> my husband . </s> _PAD']
lines size: 40750 
seq size: 232777


In [103]:
# sample the original sentence
print(getty_list[123])

seqs = [create_seq(i) for i in getty_list if i != '_']

# sample the sentence separated text
print(seqs[123])

# merge list-of-lists into a single list
seqs = sum(seqs, [])

# count of sequences
print(f'lines size: {len(getty_list)} \nseq size: {len(seqs)}')

<s> very well .   </s>
['<s> very well . </s> _PAD']
lines size: 3426 
seq size: 17404


**Checkpoint 4:**

Create inputs and targets (x and y) for training; a sample of your x and y should look like the following. Remember to pad sequences with less than 5 tokens. (NOTE: the sample is just a pair of x,y, not all x's /y's for inputs and targets).

In [104]:
# create inputs and targets (x and y)
x = []
y = []

for s in seqs:
    x.append(" ".join(s.split()[:-1]))
    y.append(" ".join(s.split()[1:]))

# print(f'sequence: {seqs[12]}')
print(f'sample x: {x[12]}')
print(f'sample y: {y[12]}')

sample x: the confederate army of northern
sample y: confederate army of northern virginia


**Checkpoint 5:**

Compute a list of all words that need to be assigned as UNK. Part of the unknown list should look like the following.

In [109]:
# create integer-to-token mapping
int2token = {}
tok_cnt = 0
singleUNK = []

tokList = " ".join(getty_list).split()
for w in set(" ".join(getty_list).split()):
    if tokList.count(w) > 1:
        int2token[tok_cnt] = w
        tok_cnt+= 1
    else:
        singleUNK.append(w)

# create token-to-integer mapping
token2int = {t: i for i, t in int2token.items()}

set_singleUNK = set(singleUNK)
print(f' # of singletons {len(singleUNK)}')
# testing to match the provided answer
  # test original sentence list for count
print(f' Test for singleUNK  defended {" ".join(sent_list).split().count("defended")==1}')
  # test identified singleUNK tokens
print(f" Test for singleUNK  unwearying {singleUNK.count('unwearying')==1}, although {singleUNK.count('although') == 1}")
# list of single words assigned to UNK
print(singleUNK)

 # of singletons 909
 Test for singleUNK  defended False
 Test for singleUNK  unwearying False, although False
['stops', 'irony', 'hereby', 'newborn', 'political', 'privacy', 'respectfully', 'reforming', 'arthur', 'solid', 'foe', 'ends', 'emissary', 'civil', 'saying', 'martial', 'civilized', 'thick', 'canister', 'billy', 'killer', 'dream', 'marching', 'scareheads', 'requested', 'guardian', 'cry', 'dogs', 'pea', 'touch', 'eccentric', 'climbing', 'yard', 'directed', 'ethical', 'daddy', 'enfilade', 'reverence', 'someplace', 'price', 'crown', 'commit', 'weeks', 'thump', 'replenished', 'assure', 'crowning', 'posts', 'leading', 'lengthening', 'act', 'bored', 'black', 'passes', 'air', 'amazing', 'fingers', 'himself', 'fortified', 'changed', 'further', 'reconnaissance', 'stranger', 'reinforce', 'bucko', 'walked', 'debt', 'bandage', 'disorganized', 'heart', 'aimed', 'heck', 'waked', 'expected', 'johnston', 'agree', 'denied', 'wives', 'rifle', 'surprised', 'faculties', 'earnestly', 'impression',

In [105]:
# create integer-to-token mapping
int2token = {}
tok_cnt = 0
singleUNK = []

tokList = " ".join(sent_list).split()
for w in set(" ".join(sent_list).split()):
    if tokList.count(w) > 1:
        int2token[tok_cnt] = w
        tok_cnt+= 1
    else:
        singleUNK.append(w)

# create token-to-integer mapping
token2int = {t: i for i, t in int2token.items()}

set_singleUNK = set(singleUNK)
print(f' # of singletons {len(singleUNK)}')
# testing to match the provided answer
  # test original sentence list for count
print(f' Test for singleUNK  defended {" ".join(sent_list).split().count("defended")==1}')
  # test identified singleUNK tokens
print(f" Test for singleUNK  unwearying {singleUNK.count('unwearying')==1}, although {singleUNK.count('although') == 1}")
# list of single words assigned to UNK
print(singleUNK)

 # of singletons 3830
 Test for singleUNK  defended False
 Test for singleUNK  unwearying False, although False
['washing', 'authentic', 'betterment', 'gains', 'barren', 'reside', 'lumbering', 'privacy', 'respectfully', 'flows', 'benevolent', 'enslavers', 'coldest', 'sobriety', 'readying', 'thugs', 'blushing', 'nepoticide', 'foe', 'abated', 'rookery', 'invaluable', 'eternally', 'stump', 'chapter', 'swallows', 'rested', 'eater', 'surest', 'shite', 'shirei', 'clue', 'which-', 'mulish', 'tribute', 'salacious', 'castration', 'bum', 'chip', 'administering', 'churned', 'loathe', 'mice', 'someplace', 'mink', 'chieftain', 'attractive', 'ascertain', 'volcanic', 'neila', 'le-', 'capacity', 'unconnected', 'bony', 'retrieving', 'mainlanders', 'tulip', 'arakh', 'windy', 'silvers', 'barbaro', 'disr-', 'suddenly', 'snowing', 'spurs', 'terrifies', 'irrefutable', 'mug', 'moss', 'ingrate', 'added', 'chrissen', 'briefly', 'vouched', 'aged', 'slows', 'anvil', 'dictate', "isn't-", 'browning', 'chewy', 'swa

['unwearying', 'defended', 'pointedly', 'although', 'utterly', 'resisted', 'gratification', 'chimney', 'digressions', 'improvements', 'retail', 'imitations', 'china', 'broad', 'faced', 'stuffy', 'breathing', 'port', 'commonest', 'dullest', 'topic', 'speaker', 'rivals', 'muffin', 'life--', 'compliance', 'engrossing', 'talker', 'bets', 'prizes', 'demands', 'hesitating', 'anywhere', 'estimated', 'imposing', 'proclaim', 'breathed', 'truest', 'verily', 'topics', 'military', 'bequeathed', 'gift', 'godfather', 'heavens', 'disregarded', 'legal', 'redress', 'informality', 'bequest', 'conditional', 'forfeited', 'disgraced', 'irritated', 'inhumanity', 'boasting', 'implacability', 'resentments', 'amiable"--but', 'childhood', 'inmates', 'parental', 'confidential', 'superintendence', 'providing', 'dishonest', 'dishonesty', 'impulses', 'degenerate', 'popular', 'guardian', 'superintends', 'trials', 'reverting', 'conversible', 'equals', 'chances', 'shillings', 'regarding', 'misleads', 'arrogant', 'dict

**Checkpoint 6:**

Build two dictionaries (vocabulary), one maps a unique integer to a unique token. The other one is a mapping from the reversed direction. The following is an example of the key-value pairs of the two dictionaries. Your dictionaries should have similar formats.

In [110]:
print(f"key: 'the' value: {token2int['the']}")
print(f"key: {int2token[1244]} value: 1244")
print(f"size of the dictionaries (vocabulary): {len(int2token)} and {len(token2int)}")

key: 'the' value: 85
key: yankee value: 1244
size of the dictionaries (vocabulary): 1311 and 1311


In [106]:
print(f"key: 'the' value: {token2int['the']}")
print(f"key: {int2token[1244]} value: 1244")
print(f"size of the dictionaries (vocabulary): {len(int2token)} and {len(token2int)}")

key: 'the' value: 82
key: beetle value: 1244
size of the dictionaries (vocabulary): 5902 and 5902


key: 'the' value:  1502
key: 1327, value:  the
size of the dictionaries (vocabulary): 3907 and 3907


**Checkpoint 7:**

Add unknown and padding to the two dictionaries (check). The size of them should increase by 2.

In [111]:
# add unknown and padding tokens
int2token[tok_cnt] = '_UNK'
tok_cnt+=1
int2token[tok_cnt] = '_PAD'

# re-create token-to-integer mapping
token2int = {t: i for i, t in int2token.items()}

print(f"vocabulary sizes: {len(int2token)},{len(token2int)}")
print(f"token2int['_PAD'] {token2int['_PAD']}")
print(f"token2int['_UNK'] {token2int['_UNK']}")
vocab_size = len(int2token)

vocabulary sizes: 1313,1313
token2int['_PAD'] 1312
token2int['_UNK'] 1311


In [108]:
# add unknown and padding tokens
int2token[tok_cnt] = '_UNK'
tok_cnt+=1
int2token[tok_cnt] = '_PAD'

# re-create token-to-integer mapping
token2int = {t: i for i, t in int2token.items()}

print(f"vocabulary sizes: {len(int2token)},{len(token2int)}")
print(f"token2int['_PAD'] {token2int['_PAD']}")
print(f"token2int['_UNK'] {token2int['_UNK']}")
vocab_size = len(int2token)

vocabulary sizes: 5905,5904
token2int['_PAD'] 5904
token2int['_UNK'] 5903


vocabulary sizes: 3909,3909
token2int['_PAD'] 3908
token2int['_UNK'] 3907


**Checkpoint 8:**

Convert your sequences of tokens into integers, with the dictionaries we just built. Your x's and y's should look like the following.

In [112]:
def get_integer_seq(seq):
  return [token2int[w] if w not in set_singleUNK else token2int['_UNK'] for w in seq.split()]

# convert text sequences to integer sequences
x_int = [get_integer_seq(i) for i in x]
y_int = [get_integer_seq(i) for i in y]

# convert lists to numpy arrays
x_int = np.array(x_int)
y_int = np.array(y_int)

def get_batches(arr_x, arr_y, batch_size):

    # iterate through the arrays
    prv = 0
    for n in range(batch_size, arr_x.shape[0], batch_size):
      x = arr_x[prv:n,:]
      y = arr_y[prv:n,:]
      prv = n
      yield x, y

In [113]:
print(f'x in integers: {x_int[0:6]}')
print(f'y in integers: {y_int[0:6]}')

x in integers: [[ 574   93 1031 1311 1024]
 [  93 1031 1311 1024 1269]
 [1031 1311 1024 1269 1295]
 [1311 1024 1269 1295 1164]
 [1024 1269 1295 1164  274]
 [1269 1295 1164  274 1143]]
y in integers: [[  93 1031 1311 1024 1269]
 [1031 1311 1024 1269 1295]
 [1311 1024 1269 1295 1164]
 [1024 1269 1295 1164  274]
 [1269 1295 1164  274 1143]
 [1295 1164  274 1143  309]]


x in integers: 
[[2514 3142 1934 1341 3284]
 [3142 1934 1341 3284 2482]
 [1934 1341 3284 2482  645]
 ...
 [1002 1502 1824  148 3907]
 [1502 1824  148 3907  533]
 [1824  148 3907  533  322]]

y in integers: 
[[3142 1934 1341 3284 2482]
 [1934 1341 3284 2482  645]
 [1341 3284 2482  645 3167]
 ...
 [1502 1824  148 3907  533]
 [1824  148 3907  533  322]
 [ 148 3907  533  322 1356]]


**Checkpoint 9:**

Build your LSTM model with Pytorch. Your LSTM structure should look like the following:



In [114]:
class WordLSTM(nn.Module):

    def __init__(self, n_hidden=256, n_layers=4, drop_prob=0.3, lr=0.001):
        super().__init__()

        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr

        self.emb_layer = nn.Embedding(vocab_size, 200)

        ## define the LSTM
        self.lstm = nn.LSTM(200, n_hidden, n_layers,
                            dropout=drop_prob, batch_first=True)

        ## define a dropout layer
        self.dropout = nn.Dropout(drop_prob)

        ## define the fully-connected layer
        self.fc = nn.Linear(n_hidden, vocab_size)

    def forward(self, x, hidden):
        ''' Forward pass through the network.
            These inputs are x, and the hidden/cell state `hidden`. '''
       ## pass input through embedding layer
        embedded = self.emb_layer(x)

        ## Get the outputs and the new hidden state from the lstm
        lstm_output, hidden = self.lstm(embedded, hidden)

        ## pass through a dropout layer
        out = self.dropout(lstm_output)

        #out = out.contiguous().view(-1, self.n_hidden)
        out = out.reshape(-1, self.n_hidden)

        ## put "out" through the fully-connected layer
        out = self.fc(out)

        # return the final output and the hidden state
        return out, hidden


    def init_hidden(self, batch_size):
        ''' initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data

        # if GPU is available   mps is for mac code
        if (torch.cuda.is_available()):
          hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                    weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())

        # if GPU is not available
        else:
          hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                    weight.new(self.n_layers, batch_size, self.n_hidden).zero_())

        return hidden

In [115]:
# instantiate the model
net = WordLSTM()

# push the model to GPU (avoid it if you are not using the GPU)
dtype = torch.float
device = torch.device("cuda")
net.cuda()

print(net)

WordLSTM(
  (emb_layer): Embedding(1313, 200)
  (lstm): LSTM(200, 256, num_layers=4, batch_first=True, dropout=0.3)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=1313, bias=True)
)


WordLSTM(
  (emb_layer): Embedding(3909, 200)
  (lstm): LSTM(200, 256, num_layers=4, batch_first=True, dropout=0.3)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=3909, bias=True)
)


**Checkpoint 10:**

Train your model with the processed data.

In [116]:
def train(net, epochs=10, batch_size=32, lr=0.001, clip=1, print_every=32):

    # optimizer
    opt = torch.optim.Adam(net.parameters(), lr=lr)

    # loss
    criterion = nn.CrossEntropyLoss()

    # push model to GPU
    net.cuda()

    counter = 0

    net.train()

    for e in range(epochs):

        # initialize hidden state
        h = net.init_hidden(batch_size)

        for x, y in get_batches(x_int, y_int, batch_size):
            counter+= 1

            # convert numpy arrays to PyTorch arrays
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)

            # push tensors to GPU
            inputs, targets = inputs.cuda(), targets.cuda()

            # detach hidden states
            h = tuple([each.data for each in h])
            # zero accumulated gradients
            net.zero_grad()

            # get the output from the model
            output, h = net(inputs, h)

            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(-1))

            # back-propagate error
            loss.backward()

            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)

            # update weigths
            opt.step()

            if counter % print_every == 0:

              print("Epoch: {}/{}...".format(e+1, epochs),
                    "Step: {}...".format(counter))

In [117]:
# train the model
train(net, batch_size = 32, epochs=50, print_every=256)
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()
output_dir = './model_save/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)
torch.save(net.state_dict(), output_dir+'net_model_all.mdl')

Epoch: 1/50... Step: 256...
Epoch: 1/50... Step: 512...
Epoch: 2/50... Step: 768...
Epoch: 2/50... Step: 1024...
Epoch: 3/50... Step: 1280...
Epoch: 3/50... Step: 1536...
Epoch: 4/50... Step: 1792...
Epoch: 4/50... Step: 2048...
Epoch: 5/50... Step: 2304...
Epoch: 5/50... Step: 2560...
Epoch: 6/50... Step: 2816...
Epoch: 6/50... Step: 3072...
Epoch: 7/50... Step: 3328...
Epoch: 7/50... Step: 3584...
Epoch: 8/50... Step: 3840...
Epoch: 8/50... Step: 4096...
Epoch: 9/50... Step: 4352...
Epoch: 9/50... Step: 4608...
Epoch: 9/50... Step: 4864...
Epoch: 10/50... Step: 5120...
Epoch: 10/50... Step: 5376...
Epoch: 11/50... Step: 5632...
Epoch: 11/50... Step: 5888...
Epoch: 12/50... Step: 6144...
Epoch: 12/50... Step: 6400...
Epoch: 13/50... Step: 6656...
Epoch: 13/50... Step: 6912...
Epoch: 14/50... Step: 7168...
Epoch: 14/50... Step: 7424...
Epoch: 15/50... Step: 7680...
Epoch: 15/50... Step: 7936...
Epoch: 16/50... Step: 8192...
Epoch: 16/50... Step: 8448...
Epoch: 17/50... Step: 8704...
Ep

**Checkpoint 11:**

Generate 10 examples of text from the model, starting from ’\<s\>’ token and ending at ’\</s\>’ token. The generated texts should look more or less natural, instead of being in a very fixed pattern or randomly generated. Some examples are shown below.

In [118]:
# predict next token
def predict(net, tkn, h=None):
  # tensor inputs
  x = np.array([[token2int[tkn]]])
  inputs = torch.from_numpy(x)
  
  # push to GPU
  inputs = inputs.cuda()

  # detach hidden state from history
  h = tuple([each.data for each in h])

  # get the output of the model
  out, h = net(inputs, h)

  # get the token probabilities
  p = F.softmax(out, dim=1).data

  p = p.cpu()

  p = p.numpy()
  p = p.reshape(p.shape[1],)

  # get indices of top 3 values
  top_n_idx = p.argsort()[-3:][::-1]

  #print(len(p))  p = vocabulary size without special tokens 

  # randomly select one of the three indices
  sampled_token_index = top_n_idx[random.sample([0,1,2],1)[0]]

  # return the encoded value of the predicted char and the hidden state
  return int2token[sampled_token_index], h


# function to generate text
def sample(net, size, prime='<s>'):
        
    # push to GPU
    net.cuda()
    
    net.eval()

    # batch size is 1
    h = net.init_hidden(1)

    toks = prime.split()

    # predict next token
    for t in prime.split():
        token, h = predict(net, t, h)
    
    toks.append(token)

    # predict subsequent tokens
    for i in range(size-1):
        token, h = predict(net, toks[-1], h)
        toks.append(token)
        if token == '</s>':
            break

    return ' '.join(toks)

In [119]:
for i in range(1):
    print(sample(net,1105))

<s> in june the _UNK " " _UNK " with the potomac to be told the top of this line is coming this army even quick today ? . - yes . </s>


In [149]:
loc_inters = ['is north of','is south of', 'is east of', 'is west of', 'is close to', 'is far from']
loc_inters_last = ['going to', 'coming from', 'the hills in', 'the trees in', 'the water in', 'the field in']
locs = ['potomac',"virginia", 'little round top','peach orchard', 'oak hill', "power 's hill"]

getty_primers = []
for i in locs:
    for j in loc_inters:
        getty_primers.append(i+' '+j)
    for k in loc_inters_last:
        getty_primers.append(k+' '+i)
print(getty_primers)

['potomac is north of', 'potomac is south of', 'potomac is east of', 'potomac is west of', 'potomac is close to', 'potomac is far from', 'going to potomac', 'coming from potomac', 'the hills in potomac', 'the trees in potomac', 'the water in potomac', 'the field in potomac', 'virginia is north of', 'virginia is south of', 'virginia is east of', 'virginia is west of', 'virginia is close to', 'virginia is far from', 'going to virginia', 'coming from virginia', 'the hills in virginia', 'the trees in virginia', 'the water in virginia', 'the field in virginia', 'little round top is north of', 'little round top is south of', 'little round top is east of', 'little round top is west of', 'little round top is close to', 'little round top is far from', 'going to little round top', 'coming from little round top', 'the hills in little round top', 'the trees in little round top', 'the water in little round top', 'the field in little round top', 'peach orchard is north of', 'peach orchard is south o

In [150]:
for primer in getty_primers:
    print(sample(net,1105,prime=primer))

potomac is north of an strength with a uncle at west justice and _UNK these own maine _UNK _UNK , but we are told you , _UNK the _UNK , but the cannons are going for double george meade , _UNK _UNK _UNK _UNK , but the yankees are . - yes , sir , i said you are relieved , captain . - no . - colonel ! </s>
potomac is south of _UNK - men batteries from all batteries to _UNK their own lines ! - yes , but i do n't have much left to be some _UNK _UNK . . . - general , give him a fault . - _UNK , boys ! march of this war ! - _UNK . . _UNK , men ! march ! . . - no . </s>
potomac is east of that hill . . _UNK _UNK the potomac , boys of _UNK lincoln and help the ridge and form up a defensive position , sir . . . . - yes to _UNK _UNK his _UNK before what they have . - captain ! </s>
potomac is west of them like we did at _UNK , but the _UNK , seven _UNK , men ? </s>
potomac is close to _UNK his _UNK and six are tired of virginia , boys . - captain spear . - yes , flankers ? </s>
potomac is far fr

#### 20 Accurate   52 InAccurate Gettysburg
Counted as "Accurate"
1) power 's hill is far from your picket line . </s>

2) the trees in power 's hill of virginia , form your _UNK . - _UNK . - brained hot is massed enough down there today for a _UNK , boys . . </s>

3) potomac is north of an strength with a uncle at west justice and _UNK these own maine _UNK _UNK , but we are told you , _UNK the _UNK , but the cannons are going for double george meade , _UNK _UNK _UNK _UNK , but the yankees are . - yes , sir , i said you are relieved , captain . - no . - colonel ! </s>

4)potomac is east of that hill . . _UNK _UNK the potomac , boys of _UNK lincoln and help the ridge and form up a defensive position , sir . . . . - yes to _UNK _UNK his _UNK before what they have . - captain ! </s>

5)potomac is far from my _UNK and firing back on that hill , _UNK _UNK . - _UNK . </s>

6)coming from potomac , robert good regiment to the top of this day , we can be _UNK , general . </s>

7)the hills in potomac . . " - i do n't suppose it is general hancock ? </s>

8)power 's hill is south of _UNK _UNK the potomac to _UNK their position in reserve ? " _UNK , but the _UNK guard . </s>

9) potomac is close to _UNK his _UNK and six are tired of virginia , boys . - captain spear . - yes , flankers ? </s>

10)the hills in peach orchard in virginia . </s>

11)the field in peach orchard in virginia ? " _UNK the _UNK . </s>

12)the field in potomac , north ! </s>

13)the water in potomac , robert guns like the opportunity , _UNK the _UNK . - yes for the union corps ? </s>

14)virginia is north of everybody are _UNK _UNK and six _UNK _UNK _UNK , _UNK mess and legs in books to _UNK their _UNK . - captain goree , sir , the _UNK _UNK , all right . - yes , sir . </s>

15)virginia is south of yankee cavalry : men 's getting thin _UNK _UNK . - brained of _UNK of his own son ! </s>

16)virginia is close to _UNK the potomac to get them _UNK or _UNK to the wall ! - brained of good man ! . </s>

17)the field in virginia , robert three army back up the ridge and _UNK , sir ! . . . - i 'll tell him , _UNK . . </s>

18) little round top is far from pickett on the _UNK of this ridge and six 's wounded , we 'll be _UNK _UNK here , sir , are to wake them there is patrick more good morning . . - yes , flankers ? </s>

19)power 's hill is north of an actor ? . . - i 'll tell me a _UNK , sir , the federal damn reb corps will converge upon them down this hill and _UNK to _UNK their own _UNK . </s>

20)peach orchard is south of northern june , robert guns to be _UNK today </s>


Counted as INACURATE:

1) the field in power 's hill 's brigades . </s>
2) the water in power 's hill has told him to _UNK their position and six do you have to use general lee , missing of virginia to get yourself _UNK . - colonel ! - yes , flankers . . . does , you 'll need your help , but i 'd known , sir ? </s>
3) the hills in power 's hill would never been a clear advantage , _UNK of his _UNK ! " year for _UNK . - _UNK . - _UNK ! march ! - major ! march as it 's nothing you 're _UNK a surgeon forget ? </s>
4) coming from power 's hill 's _UNK _UNK the artillery is out through god 's hands ! march of that road , all _UNK these head up the _UNK military family ? . - _UNK . . colonel </s>
5) going to power 's hill ? </s>
6) potomac is south of _UNK - men batteries from all batteries to _UNK their own lines ! - yes , but i do n't have much left to be some _UNK _UNK . . . - general , give him a fault . - _UNK , boys ! march of this war ! - _UNK . . _UNK , men ! march ! . . - no . </s>
7)power 's hill is east of it ! - yes for the union second corps with them to use _UNK his corps . . _UNK _UNK . - brained of the _UNK - _UNK mess . </s>
8) power 's hill is west of those _UNK . - quick ! </s>
9)power 's hill is close to the rear of the army is moving here with us , but there , as my _UNK ! . colonel </s>
10)going to potomac , sir ! _UNK the _UNK of _UNK . - yes , lo ! </s>
11)potomac is west of them like we did at _UNK , but the _UNK , seven _UNK , men ? </s>
12)peach orchard is east of them . </s>
13)peach orchard is west of the army is still very men , but i 'll be moved . . . </s>
14)peach orchard is close to _UNK the artillery ! </s>
15)peach orchard is far from your hands to _UNK _UNK . </s>
16)going to peach orchard , boys as they come up ? </s>
17)coming from peach orchard and trees , _UNK . . </s>
18)the trees in potomac to lead the old _UNK _UNK of the mountains and i 'll have . - _UNK ! march of that line , i 'll be _UNK a scrap and too getting knows that we can do that ? - i 'm sorry . - friday , july ! thank you ? . colonel </s>
19)virginia is east of this gap ? - yes , lo ! . colonel </s>
20)virginia is west of our own maine time . - _UNK . </s>
21)virginia is far from your uncle who got them at your hands , i said about your _UNK and _UNK are long batteries in virginia , men are coming , but we are moved and set up a little ways here . </s>
22)going to virginia today . . _UNK , _UNK the morning , i want it in this war , the _UNK pennsylvania . </s>
23)coming from virginia the situation out up . . _UNK for _UNK , but your king your _UNK . </s>
24)the hills in virginia , but i do appreciate him again , sir , you 've ever confess . . . </s>
25)the trees in virginia ? " general armistead is his class now they 're bringing up . . </s>
26)going to little round top , if i can tell you , sir , general , we can not be moved . </s>
27)little round top is close to support the _UNK and be an _UNK - quick ? march away . - yes , major ! - _UNK , sir , i 'm _UNK a lot , sir , are you with me , boys ! thank this day ? thank you ? </s>
28)little round top is west of here . . _UNK _UNK the blue ridge . - friday and july ? </s>
29)little round top is east of this town ? </s>
30)little round top is south of _UNK _UNK _UNK their own officer on us . - _UNK . </s>
31)the water in virginia . </s>
32)little round top is north of virginia here . - _UNK , _UNK _UNK </s>
33)coming from little round top , boys , robert corps would be _UNK . - _UNK . . _UNK and _UNK are _UNK up the narrow roads ? </s>
34)peach orchard is north of _UNK , men ! </s>
35)the field in little round top . . . </s>
36)the water in little round top _UNK the _UNK , men . </s>
37)the trees in little round top " _UNK _UNK " _UNK " . </s>
38)the hills in little round top _UNK the potomac to get out , men as possible , but i 'm going to lose a brigade , _UNK _UNK _UNK _UNK . . . </s>
39)the trees in peach orchard of _UNK _UNK , _UNK _UNK , but it was very sorry to _UNK the artillery train fuss . </s>
40)the field in oak hill has _UNK right his head from the others to keep the lead charge ! . </s>
41)the water in oak hill is _UNK " _UNK _UNK _UNK " " up there , _UNK mess of virginia , _UNK , men , but the war would go up this day . </s>
42)the trees in oak hill ? - _UNK _UNK " . - _UNK ! march is and they 're reinforced ? . </s>
43)the hills in oak hill _UNK to the last - _UNK mess . - _UNK . . colonel . </s>
44)coming from oak hill than the army is still a little army ? - friday 's happening . - _UNK . . . </s>
45)going to oak hill . " _UNK to _UNK _UNK . . . </s>
46)that 's james , _UNK , _UNK , _UNK ! </s>
47)oak hill is far from my division ? - friday is true , sir ? - sir , i 'm _UNK 
48)oak hill is close to _UNK the union _UNK here " _UNK _UNK _UNK _UNK _UNK " _UNK " and _UNK . </s>
49)the water in peach orchard and wheat _UNK " . . _UNK , _UNK file . . - yes . . . . </s>
50)oak hill is north of everything and we can _UNK their own _UNK _UNK the _UNK of me for a column of _UNK and form them down that road . " </s>
51)oak hill is west of those guns back and fired down on the left of this ridge and six 's _UNK , boys ! </s>
52)oak hill is east of it ? . </s>
53)oak hill is south of history , but i 've been ordered this job ? - i 'm going for one behind me , in virginia . . </s>

In [151]:
for primer in getty_primers:
    print(sample(net,1105,prime=primer))

potomac is north of some _UNK - _UNK _UNK . - brained bastards on an actor ? </s>
potomac is south of _UNK _UNK the _UNK of virginia . . . </s>
potomac is east of that line and the line of _UNK - _UNK _UNK . </s>
potomac is west of us , i thought it could be _UNK at us as we are the _UNK and quick today . </s>
potomac is close to _UNK his wounded . </s>
potomac is far from virginia , robert _UNK , men ! - yes , lo ? </s>
going to potomac for the _UNK _UNK of virginia , men ! </s>
coming from potomac , scout them up , but i 'm fine and wheat back to heathen opened , if we help this ridge and alabama , i do not believe we are n't going to gain some _UNK at all of the guns on me to _UNK the _UNK of virginia , men . </s>
the hills in potomac to the center and _UNK , _UNK the _UNK _UNK , all of virginia _UNK and west point is coming that _UNK _UNK , but you are _UNK _UNK ? - _UNK ! march of it , missing these boys like up here , major . . . - i thought i 'm going me , _UNK mess and missing 

In [ ]:
for primer in getty_primers:
    print(sample(net,1105,prime=primer))

In [26]:
for i in range(1):
    print(sample(net,1105))

<s> i 'm sorry i 'm not going back to king in the south ! ” it 's the only way to die . " i 'll have a baby , you 're the only person left in the city , and we have a choice , i 'll be the first of my own father , the queen of the iron islands . ” and i 'll be sure . . </s>


In [43]:
loc_inters = ['is north of','is south of', 'is east of', 'is west of', 'is close to', 'is far from']
loc_inters_last = ['going to', 'coming from', 'the sun in', 'the trees in', 'the water in', 'the sky in']
locs = ['winterfell',"king 's landing", 'harrenhal','iron islands', 'valyria', 'pentos']
make_primers = []
for i in locs:
    for j in loc_inters:
        make_primers.append(i+' '+j)
    for k in loc_inters_last:
        make_primers.append(k+' '+i)
print(make_primers)

['winterfell is north of', 'winterfell is south of', 'winterfell is east of', 'winterfell is west of', 'winterfell is close to', 'winterfell is far from', 'going to winterfell', 'coming from winterfell', 'the sun in winterfell', 'the trees in winterfell', 'the water in winterfell', 'the sky in winterfell', "king 's landing is north of", "king 's landing is south of", "king 's landing is east of", "king 's landing is west of", "king 's landing is close to", "king 's landing is far from", "going to king 's landing", "coming from king 's landing", "the sun in king 's landing", "the trees in king 's landing", "the water in king 's landing", "the sky in king 's landing", 'harrenhal is north of', 'harrenhal is south of', 'harrenhal is east of', 'harrenhal is west of', 'harrenhal is close to', 'harrenhal is far from', 'going to harrenhal', 'coming from harrenhal', 'the sun in harrenhal', 'the trees in harrenhal', 'the water in harrenhal', 'the sky in harrenhal', 'iron islands is north of', 'i

In [42]:
for primer in make_primers:
    print(sample(net,1105,prime=primer))


prime = "Winterfell is North of "
prime = "Winterfell is South of "
prime = "Winterfell is East of "
prime = "Winterfell is West of "
prime = "Winterfell is close to "
prime = "Winterfell is far from "

winterfell is north of the wall , the first men . ” the king of house targaryen , i charge him . " i 'm sure you 're the one who knows how you do . " and you were n't there , you do . </s>
winterfell is south of his life . . . . </s>
winterfell is east of the dead . " and you are the most honorable woman in the north . </s>
winterfell is west of _UNK . ” and the dragon queen of house targaryen . </s>
winterfell is close to a great warrior , and i will not have a choice ? </s>
winterfell is far from the beginning . ” i do , you 'll be dead . </s>
going to winterfell , i do . " </s>
coming from winterfell and the dead , i 'm not sure what i 've been . " </s>
the sun in winterfell and _UNK and a dragon , and you 'll be a good man , you know , but i 'm sorry . . . </s>
the trees in winterfell , but i 've seen . ! . </s>
the water in winterfell . " and the people who murdered her father . </s>
the sky in winterfell , i 'll be sure to be the queen 's bastard ? </s>
kings landing is north of 

#### 13 Accurate   59 InAccurate Gettysburg
Counted as "Accurate"
1)kings landing is east of the dead and we 're not going home , but we 'll be the first man in westeros ? ” it is the only one i ever thought to confess , but you 'll have a choice , you know . ! ” it is the best who know what i 've seen the people of westeros and their lives are defeated and _UNK . </s>

2)kings landing is close to our enemies and your father 's arms and the night is gathering that is what you want to see the king in the north . </s>

3)going to kings landing . " i do , i 'll be the first to be my queen , and the first men and the night is coming , the first of his own , the king 's landing . ” and you will never see you again ? </s>

4)the sun in kings landing , i 'll be a girl , i 'm sure you 've seen the people of westeros , the _UNK _UNK the city has become an evil man ? </s>

5)harrenhal is east of _UNK , the king 's landing , the iron fleet will make it to the red wedding 's not a queen , the queen 's hand , i 'll give her the last of the unsullied . ” i do , you know . ! </s>

6)harrenhal is close to the iron throne . </s>

7)coming from harrenhal , the queen 's army is a good man , i 'll give her a debt of all of you , i do not think i 'll do anything , i 'm not the queen , you know , i 'll have to see you . ! " i 'll never see the war , you 'll be the only ones who ever ever wanted to do it ? ! </s>

8)iron islands is south of our wall . . </s>

9)iron islands is east of house targaryen and her army , and the dead , the _UNK , i 'm sorry i 'm sure he was n't a knight . </s>

10)iron islands is west of westeros and the first of your name is the king 's landing , and the karstarks are the best who murdered the queen . ” and i 'll be a bit of the night is coming . " i do , but i do n't want it . </s>

11)iron islands is close to the iron throne and the _UNK of the dead . " and you were the only man alive in this city , but you do n't have the same . " and i 've seen a man . </s>

12)iron islands is far from a war , but we can not make it . . ” and you 're the queen , the queen , and the queen , the queen of the iron fleet , and you 'll never be a fool and i 'm going to die , and you 're going back for the night king , and i will not have you ever been with a girl . </s>

13)going to iron islands , i 'll be the first men and protector and i 'm not sure you 're not a good one ! ! , i 'm sure we can not get to the wall . ” </s>
coming from iron islands , i 'm sure you 're the queen 's hand is a good thing . " </s>



Counted as "Accurate"
winterfell is north of the wall , the first men . ” the king of house targaryen , i charge him . " i 'm sure you 're the one who knows how you do . " and you were n't there , you do . </s>
winterfell is south of his life . . . . </s>
winterfell is east of the dead . " and you are the most honorable woman in the north . </s>
winterfell is west of _UNK . ” and the dragon queen of house targaryen . </s>
winterfell is close to a great warrior , and i will not have a choice ? </s>
winterfell is far from the beginning . ” i do , you 'll be dead . </s>
going to winterfell , i do . " </s>
coming from winterfell and the dead , i 'm not sure what i 've been . " </s>
the sun in winterfell and _UNK and a dragon , and you 'll be a good man , you know , but i 'm sorry . . . </s>
the trees in winterfell , but i 've seen . ! . </s>
the water in winterfell . " and the people who murdered her father . </s>
the sky in winterfell , i 'll be sure to be the queen 's bastard ? </s>
kings landing is north of the reach , i do not know that ? . </s>
kings landing is south of our family . " </s>
kings landing is west of the world and i 'm not going north of this war , and i 'll give her to you , you 'll be a good person , i 'll give it to her . . bran is the one who murdered her father , i do n't think i 'd have to make peace . </s>

kings landing is far from this city . </s>

coming from kings landing and i will never be . ” </s>

the trees in kings landing , i do . </s>
the water in kings landing . ” </s>
the sky in kings landing . ” </s>

harrenhal is north of his own , i 'm sure he 's the one . " i do n't want it . . . " i 'll never be the only person left , the unsullied are dead ! " </s>
harrenhal is south of our wall and the dead will die . ! " i do n't want it . </s>

harrenhal is west of a great house . " </s>

harrenhal is far from a distance , but we need to go home . </s>
going to harrenhal . ” </s>

the sun in harrenhal , i 'll be a good man ! </s>
the trees in harrenhal . " and the people who know how you feel , but you do n't know what i 'm doing , but we 're going home , i 'll be here soon , but i do n't know what you 've done to me , i 'll be dead . ! " i do , you know , i 'm sure you 're not here to be the last . </s>
the water in harrenhal , you 'll have you ever seen it ? " and i 'm going to kill you , you 're a good man ! ! </s>
the sky in harrenhal . ” and i 'll do anything . . . . ! ! ! ! </s>
iron islands is north of his way back back , i 'm sure i 'll give you a debt that was a terrible place in this castle , but i do n't know , i 'd have a good compromise , but i 'm sure he 's a man who has never fallen , but we 'll be dead . ” </s>


the sun in iron islands , but he 'll be the only ones in this castle ? " and you will not have a choice ? . " i 'm sure i 'll be the only person in this city , the _UNK will never be . ” and i 'm sure he 's not a good person . ” , you 're the only man who ever wanted to see you . </s>
the trees in iron islands , the _UNK of the seven kingdoms . " </s>
the water in iron islands and i will not be a knight . " </s>
the sky in iron islands . ” </s>
valyria is north of his way back with you ? </s>
valyria is south of our family , i 'll give her the iron throne , i do n't know what she wants to be the last thing i 'm not . </s>
valyria is east of men and northmen . " </s>
valyria is west of the dead . ” and the _UNK , i do n't know what it 's . . " </s>
valyria is close to the seven kingdoms and the iron fleet and i 'll give her a trial , i 'll kill him . " and the first of my house . </s>
valyria is far from the same one to keep it out of a war ? " </s>
going to valyria . . " i do n't want you . . </s>
coming from valyria to kill you . </s>
the sun in valyria and i 'll give them something . . . . " i do not know that , i do not know what it 's . " </s>
the trees in valyria . </s>
the water in valyria to kill you ? </s>
the sky in valyria to be . ” </s>
pentos is north of his heart , the _UNK , the first of his own . " and i 'll have you . </s>
pentos is south of the world ? </s>
pentos is east of house tyrell . </s>
pentos is west of a _UNK . " it 's all right . ” </s>
pentos is close to the north and we need to be . </s>
pentos is far from the citadel . ” </s>
going to pentos . </s>
coming from pentos . </s>
the sun in pentos and _UNK the world is not the most powerful man i can ever see you again , i 'm not sure you 're not going back there . ” </s>
the trees in pentos . " i do n't think it was the only one in a thousand eyes and _UNK the city . </s>
the water in pentos . ” it was a good thing . " and i 'll never get to be aware , and the world has been a terrible man , but you 'll have to choose . " </s>
the sky in pentos , the queen of house targaryen , first , and i will have you changed the bells and _UNK , _UNK , massacres , unbroken ! ” . </s>

In [44]:
for primer in make_primers:
    print(sample(net,1105,prime=primer))

winterfell is north of his way back to the throne , you 're not the one true heir . ” it is a good place . " and the first men , protector . ” . </s>
winterfell is south of the world , i do not want it to protect her ? . </s>
winterfell is east of _UNK . " and i 'll be a man who has fallen for a reason ? </s>
winterfell is west of the dead and i will never see the night 's watch now , you 'll be a bit of self and full of terror , the _UNK _UNK of _UNK , and i will be . </s>
winterfell is close to her father 's head , and i 'm sorry i 'll give him my _UNK , and i will not be the last . " i 've seen a white walker . " </s>
winterfell is far from the same . " and i 'm going back . </s>
going to winterfell , but we can do it again ? " </s>
coming from winterfell , but you 're a good person ? </s>
the sun in winterfell . </s>
the trees in winterfell ? ” </s>
the water in winterfell , you 're a good friend , the _UNK _UNK of a tyrant , the first men , lord of the iron islands . " </s>
the sk

**Task 2:**

Compute and report the perplexity of the saved model on test_1.txt and test_2.txt files. Note that the test files are already preprocessed. Compute the complexity of the test_2.txt by saving the model first and then loading it.

**Checkpoint 1:**

Load your test_1.txt file in lines as you did in task 1. The results should look similar to the following.

In [ ]:
def return_lines(fname):
    for filename in findFiles('/content/drive/MyDrive/Colab Notebooks/CS_505_PA6/'+ fname):
        novel = os.path.splitext(os.path.basename(filename))
        all_categories.append(novel)
        o_lines = readLines(filename)
    return o_lines

t1_lines = return_lines('test_1.txt')
t2_lines = return_lines('test_2.txt')

In [ ]:
print('TEST ONE')
print(random.sample(t1_lines, n, ))

print('TEST TWO')
print(random.sample(t2_lines, n))

TEST ONE
['<s> it is a very large one , i know ; and when i come to see you , i hope you will have new - furnished it , for it wanted it very much when i was there six years ago . " </s>', '<s> to marianne , he merely bowed and said nothing . </s>', '<s> i am sure you must have heard of her before . </s>', '<s> they gave themselves up wholly to their sorrow , seeking increase of wretchedness in every reflection that could afford it , and resolved against ever admitting consolation in future . </s>', '<s> " it is enough , " said she ; " to say that he is unlike fanny is enough . </s>']
TEST TWO
['<s> @nickkkjonasss i like poison ivy the best </s>', '<s> " has 0 g of milky bar left , and around 200ml of coke " </s>', '<s> " so rylee , grace ... wana go steve \'s party or not ? ? sadly since its easter i wnt b able 2 do much   but ohh well ..... " </s>', "<s> http://xlisaatown.myminicity.com/   & lt;3 now i 'm tha bomb </s>", '<s> " @cliff_forster yeah , that does work better than just wa

['<s> the family of dashwood had long been settled in sussex . </s>', '<s> their estate was large , and their residence was at norland park , in the centre of their property , where , for many generations , they had lived in so respectable a manner as to engage the general good opinion of their surrounding acquaintance . </s>', '<s> the late owner of this estate was a single man , who lived to a very advanced age , and who for many years of his life , had a constant companion and housekeeper in his sister . </s>', '<s> but her death , which happened ten years before his own , produced a great alteration in his home ; for to supply her loss , he invited and received into his house the family of his nephew mr. henry dashwood , the legal inheritor of the norland estate , and the person to whom he intended to bequeath it . </s>', "<s> in the society of his nephew and niece , and their children , the old gentleman 's days were comfortably spent . </s>", '<s> his attachment to them all incre

**Checkpoint 2:**

Compute the perplexity on test_1.txt. Your final perplexity should look like the one below. (You may need to implement the perplexity calculation on your own instead of using any existing ones from libraries.)

In [ ]:
# create a dict of all words in test_1 tok and int

# create a list of unk words in test  map to _UNK

# send predict every word in test set and retrieve next word for actual next word token #

# add probabilities together for all words divide by N (vocab len including toks) and log 2.


# predict next token
def pp_next(net, tkn, target_tok, h=None):
         
  # tensor inputs
  x = np.array([[token2int[tkn]]])
  inputs = torch.from_numpy(x)
  
  # push to GPU
  inputs = inputs.cuda()

  # detach hidden state from history
  h = tuple([each.data for each in h])

  # get the output of the model
  out, h = net(inputs, h)

  # get the token probabilities
  p = F.softmax(out, dim=1).data

  p = p.cpu()

  p = p.numpy()
  p = p.reshape(p.shape[1],)

  # get indices of top 3 values
  top_n_idx = p.argsort()[-3:][::-1]

  # randomly select one of the three indices
  sampled_token_index = top_n_idx[random.sample([0,1,2],1)[0]]

  # return the encoded value of the predicted char and the hidden state
  return int2token[sampled_token_index], avg_perp=0
len_test = 0
while True:
    try:
        t = next(test)
    except StopIteration:
        break
    perp_score = model1.perplexity(t)
    avg_perp+= perp_score
    len_test +=1

    # print(' '.join(test_nft_list_1f[0]))
    # print(perp_score)
    # temp = test_nft_list_1f.pop(0)

perplexity on test_1.txt is:  172.75867345155558


**Checkpoint 3:**

Save your model with the 'state_dict()' method. Load your saved model and sample some generated texts to see if it works properly. The sample texts could look like the following one.

In [59]:
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()
output_dir = './model_save/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)
torch.save(net.state_dict(), output_dir+'net_model.mdl')

Saving model to ./model_save/


In [60]:
!ls -l --block-size=K ./model_save/

total 37092K
-rw-r--r-- 1 root root 18543K Dec 14 09:14  net_model.mdl
-rw-r--r-- 1 root root 18545K Dec 14 09:07 'WordLSTM('$'\n''  (emb_layer): Embedding(5904, 200)'$'\n''  (lstm): LSTM(200, 256, num_layers=4, batch_first=True, dropout=0.3)'$'\n''  (dropout): Dropout(p=0.3, inplace=False)'$'\n''  (fc): Linear(in_features=256, out_features=5904, bias=True)'$'\n'').bin'


In [61]:
!ls -l --block-size=M ./model_save/net_model.mdl

-rw-r--r-- 1 root root 19M Dec 14 09:14 ./model_save/net_model.mdl


In [62]:
# Copy the model files to a directory in your Google Drive.
!cp -r ./model_save/ "./WLSTM-Fine-Tuning/"

In [70]:
# Load a trained model and vocabulary that you have fine-tuned
output_dir = './model_save/'


net = WordLSTM()
net.load_state_dict(torch.load(output_dir+'net_model.mdl'))
net.eval()
# tokenizer = BertTokenizer.from_pretrained(output_dir)

# Copy the model to the GPU.
net.cuda()

WordLSTM(
  (emb_layer): Embedding(5904, 200)
  (lstm): LSTM(200, 256, num_layers=4, batch_first=True, dropout=0.3)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=5904, bias=True)
)

In [71]:
for i in range(1):
    print(sample(net,1105, prime='<s>'))

<s> i do not know what it means , but you do n't know what i want . " and the _UNK . " </s>


'<s> " oh , yes ! pray be _UNK , i am afraid of her , she could not be insensible to the latter method , " said elizabeth ; and she had no reason , i am sure i have not a little mistake , and the _UNK _UNK of his _UNK , she could not help cautioning her to see her ; for she was not in a very different _UNK . " </s>'

**Checkpoint 4:**

Load your test_2.txt file in lines. Compute the perplexity with your loaded model. The perplexity should look close to (or better than) the following one.

In [ ]:
# run perplexity on test_2

perplexity on test_2.txt is:  951.2221688812576


**Task 3:**

Train the language model as before, but with input sequence lengths of 25 (currently, its inputs are of length 5).
You may need to pad some of the shorter sentences to length 25. Save your trained model. Generate 10 examples of text
from it, starting from ’\<s\>’ token and ending at ’\</s\>’ token. **Answer in a separate cell:** Are there differences from the generated examples from
2.1? 

**Task 4:**
Compute and report the perplexity of this saved model on test 1.txt file. **Answer in a separate cell:** Is there any reason why this model is better/worse than the previous one (length = 5)?

NOTE: at this point you should be able to finish the tasks without the checkpoints, try to re-use the code you've implemented.

In [ ]:
# change to 25 and rerun

In [ ]:
# run perplexity on new model

**Task 5:**

Train the better language model as before but start with pre-trained Glove6B 100d embeddings (see [here](https://medium.com/@martinpella/how-to-use-pre-trained-word-embeddings-in-pytorch-71ca59249f76) on how
to incorporate pretrained embeddings in your LSTM model). This time, use all your words in the corpus as vocabulary,
even those occurring only once in the corpus. Only assign UNK token to words that are not in the Glove vocabulary and
initialize random vectors in the embedding matrix for the UNK, \<s\>, \</s\>, and PAD tokens using a Gaussian
distribution with mean 0.0 and σ set to 0.6 (see numpy.random.normal). Save your trained model. Generate 10 examples of text from it,
starting from the ’\<s\>’ token and ending at the ’\</s\>’ token. Are there differences from the generated examples from before?

**Task 6:**
Compute and report the perplexity of this saved model on the test 1.txt file.

NOTE: 

1. The biggest difference between this task and the previous starts from the word-integer dictionary. In addition to the mappings between word and integer, you also need to have a dictionary that maps the word to the word vector in Glove. Don't forget to add UNK, PAD, \<s\> and \</s\> to the dictionary. As these tokens do not exist in Glove, you need np.random.normal(...) to assign them random values.
2. The next difference is since the most of the word embeddings are pre-trained, you need to initialize nn.Embed with the Glove embedding matrix. You also need to adjust the model to fit the embedding as well.
3. The training might take a very long time, make sure you have your GPU utilized for Pytorch and get enough time running it.

In [ ]:
# change embedding

In [ ]:
# run perplexity